<a href="https://colab.research.google.com/github/Nasuz092/Dataset-Twitter/blob/main/Cleaning_buat_VOSViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import modul
import pandas as pd
import numpy as np
import regex as re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/Nasuz092/Trial/refs/heads/main/youtube-comments.csv', sep=',')
df .head(3)

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2024-11-10T02:07:51Z,@ploopyanimations,We would like to thank BNI for this great oppo...,6098.0
1,2024-11-10T03:15:40Z,@muhammadguntoro753,"why there is no credit at the end of video, th...",156.0
2,2024-11-10T03:23:12Z,@brezzendorf,"​@@muhammadguntoro753not necessarily, it depen...",412.0


In [8]:
# Menghapus Kolom yang tidak digunakan
df = df.drop(columns=['publishedAt',	'authorDisplayName'], axis=1)

# Menghapus data yang memiliki nilai NaN dalam kolom tertentu
df = df.dropna(subset=['textDisplay',	'likeCount'])

# Menghapus nilai yang Duplikat atau sama persis
df = df.drop_duplicates(subset=['textDisplay'])
df

,textDisplay,likeCount
0,We would like to thank BNI for this great oppo...,6098.0
1,"why there is no credit at the end of video, th...",156.0
2,"​@@muhammadguntoro753not necessarily, it depen...",412.0
3,Rimsvek buat kalian 🫡,43.0
4,Mantaap weh,42.0
...,...,...
3247,Ini yg bikin studio lokal mang<br>Cuman sayang...,1.0
3248,Wow amazing 😍😍😍🎉🎉🎉,2.0
3249,Work of art🎉😂👏,2.0
3250,Can’t wait for Episode 2 🎉,9.0


In [9]:
import re
import unicodedata

def remove_pattern(Text, pattern):
    r = re.findall(pattern, str(Text))
    for i in r:
        Text = re.sub(i, '', str(Text))
    return Text
df['remove_user'] = np.vectorize(remove_pattern)(df['textDisplay'], "@[\w]*")

def cleaning(Text):
    Text = re.sub(r"<a\s+href='https:\/\/www\.youtube\.com\/watch\?v=[^']+(&t=[0-9ms]+)?'>[^<]+<\/a>", "", Text)
    Text = re.sub(r"<a\s+href='(?:https?:\/\/)?(?:www\.)?youtube\.com[^']*(&t=[0-9ms]+)?'>[^<]+<\/a>", "", Text)
    Text = re.sub(r"<br\s*\/?>", "", Text)
    Text = Text.replace("&#39;", "")
    Text = re.sub(r"&lrm;", "", Text)
    Text = re.sub(r"&zwnj;", "", Text)
    Text = re.sub(r'\s&amp\s', ' ', Text)
    Text = re.sub(r'$\w*', '', Text)
    Text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', Text)
    Text = re.sub('"'," ", Text)
    Text = re.sub(r"\d+", " ", str(Text))
    Text = re.sub(r"[^\w\s]", " ", str(Text))
    Text = re.sub(r"\s+", " ", str(Text))
    Text = re.sub(r'#', '', Text)
    Text = re.sub(r'[^a-zA-z0-9]', ' ', str(Text))
    Text = re.sub(r'\s\s+', ' ', Text)
    Text = re.sub(r'^b[\s]+', '', Text)
    Text = re.sub(r'^link[\s]+', '', Text)
    Text = re.sub(r"<\/?[a-z]+>", "", Text)
    Text = re.sub(r"<a\s+[^>]*>", "", Text)
    return Text

def remove_emoji(Text):
    emoji = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # simbol & piktogram
                        u"\U0001F680-\U0001F6FF"  # transportasi & simbol peralatan
                        u"\U0001F1E0-\U0001F1FF"  # bendera negara
                        u"\U00002702-\U000027B0"  # simbol
                        u"\U000024C2-\U0001F251"  # emoji lainnya
                        "]+", flags=re.UNICODE)
    return emoji.sub(r'', Text)

df['cleaning'] = df['remove_user'].apply(cleaning)

In [10]:
def tokenize_with_quotes(text):
    tokens = word_tokenize(text)
    tokens_with_quotes = ['"' + token + '"' for token in tokens]
    return tokens_with_quotes

df['tokenized_Content'] = df['textDisplay'].apply(lambda x: tokenize_with_quotes(x))
df.head(6)

,textDisplay,likeCount,remove_user,cleaning,tokenized_Content
0,We would like to thank BNI for this great oppo...,6098.0,We would like to thank BNI for this great oppo...,We would like to thank BNI for this great oppo...,"[""We"", ""would"", ""like"", ""to"", ""thank"", ""BNI"", ..."
1,"why there is no credit at the end of video, th...",156.0,"why there is no credit at the end of video, th...",why there is no credit at the end of video the...,"[""why"", ""there"", ""is"", ""no"", ""credit"", ""at"", ""..."
2,"​@@muhammadguntoro753not necessarily, it depen...",412.0,"​muhammadguntoro753not necessarily, it depends...",muhammadguntoro not necessarily it depends on...,"[""​"", ""@"", ""@"", ""muhammadguntoro753not"", ""nece..."
3,Rimsvek buat kalian 🫡,43.0,Rimsvek buat kalian 🫡,Rimsvek buat kalian,"[""Rimsvek"", ""buat"", ""kalian"", ""🫡""]"
4,Mantaap weh,42.0,Mantaap weh,Mantaap weh,"[""Mantaap"", ""weh""]"
5,​@@muhammadguntoro753 biasanya kalau udah ikla...,1.0,​muhammadguntoro753 biasanya kalau udah iklan ...,muhammadguntoro biasanya kalau udah iklan ema...,"[""​"", ""@"", ""@"", ""muhammadguntoro753"", ""biasany..."


In [11]:
df.isnull().sum()

,0
textDisplay,0
likeCount,0
remove_user,0
cleaning,0
tokenized_Content,0


In [12]:
# Menghapus Kolom yang tidak digunakan
df = df.drop(columns=['likeCount',	'remove_user', 'cleaning', 'tokenized_Content'], axis=1)

# Menghapus data yang memiliki nilai NaN dalam kolom tertentu
df = df.dropna(subset=['textDisplay'])

# Menghapus nilai yang Duplikat atau sama persis
df = df.drop_duplicates(subset=['textDisplay'])
df

,textDisplay
0,We would like to thank BNI for this great oppo...
1,"why there is no credit at the end of video, th..."
2,"​@@muhammadguntoro753not necessarily, it depen..."
3,Rimsvek buat kalian 🫡
4,Mantaap weh
...,...
3247,Ini yg bikin studio lokal mang<br>Cuman sayang...
3248,Wow amazing 😍😍😍🎉🎉🎉
3249,Work of art🎉😂👏
3250,Can’t wait for Episode 2 🎉


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3151 entries, 0 to 3251
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   textDisplay  3151 non-null   object
dtypes: object(1)
memory usage: 49.2+ KB


In [14]:
#Save Data yag sudah di cleasing ke gdrive
df.to_csv('WondrBNI2024.csv', index=False)

In [17]:
from google.colab import files

# Ganti nama filenya dengan nama file yang sudah di save di gdrive sebelumnya.
files.download('WondrBNI2024.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>